In [ ]:
import os
from pprint import pprint, pformat
from concurrent.futures import (
    ThreadPoolExecutor, 
    as_completed
)
from datetime import datetime

import ndjson
from requests import RequestException

from ncpi_fhir_utility.client import FhirApiClient

FHIR_API = os.getenv('FHIR_API') or 'http://localhost:8000'
FHIR_USER = os.getenv('FHIR_USER') or 'admin'
FHIR_PW = os.getenv('FHIR_PW') or 'password'

DATA_DIR = './synthetic-data'
# Add a target profile(s) below
PROFILES = [
    'Practitioner', # 5898
    'Organization', # 5898
    'Patient', # 4967
    'Encounter', # 62571
    'Condition', # 26984
    'Observation', # 202064
    'DiagnosticReport', # 23133
]
BASE_PROFILE_URL = 'http://hl7.org/fhir/StructureDefinition'

In [ ]:
# Initialize FHIR API client
client = FhirApiClient(base_url=FHIR_API, auth=(FHIR_USER, FHIR_PW))

In [ ]:
def send_resource(resource):
    api_path = f'{FHIR_API}/{profile}'
    verb = 'POST'
    if 'id' in resource:
        api_path = f'{api_path}/{resource["id"]}'
        verb = 'PUT'

    success, result = client.send_request(
        verb, api_path, json=resource
    )
    return success, result, resource


def consume_futures(futures):
    for future in as_completed(futures):
        success, result, resource = future.result()
        # print(f'Sent {result["response"]["id"]}')
        if success:
            '''
            print(f'{result["response"]["id"]}')
            '''
            continue
        else:         
            raise Exception(
                f'Failed to submit:\n{pformat(resource)}'
                f'\nGot:\n{pformat(result)}'
            )

In [ ]:
# Load FHIR resources
begin = datetime.now()
for state in os.listdir(DATA_DIR):
    if state.startswith('.'):
        continue

    print(f'Begin loading resources from {state}')
    base_path = f'{DATA_DIR}/{state}/fhir'
    
    for profile in PROFILES:
        file_path = os.path.join(
            base_path, 
            f'{profile}.ndjson'
        )
        
        resources = ndjson.load(open(file_path))
        print(f'  Loading {profile} resources')
        with ThreadPoolExecutor(max_workers=10) as tpex:
            futures = []
            for resource in resources:
                # Set base profile
                resource \
                    .setdefault('meta', {}) \
                    .setdefault('profile', []) \
                    .append(f'{BASE_PROFILE_URL}/{profile}')
                futures.append(
                    tpex.submit(send_resource, resource)
                )
            consume_futures(futures)
    print(f'Done loading resources from {state}')

print(f'✅ Took {datetime.now() - begin}')